# Assignment - Binary classification on the Leukemia dataset using Elastic Net regularization

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Loading and pre-processing of the dataset

In this project we will use the ```leukemia``` dataset from the ```sklearn``` library, a high-dimensional dataset of genetic data where $p >> n$. More precisely, the dataset consists of $72$ observations and $7129$ features. Our goal is to perform binary classification using these features to determine the type of leukemia of each patient. It is natural to chose regularized logistic regression models to handle this task. Let us first load and have a look at the data.

In [2]:
print("Loading data...")
dataset = fetch_openml("leukemia")

X = dataset.data.astype(float)
y = (dataset.target!='ALL').astype(int)
print('Done !')
print('n = {}, p = {}'.format(X.shape[0], X.shape[1]))

Loading data...
Done !
n = 72, p = 7129


In general when training a supervised model, the first thing to do is to split our dataset into train, validation and test subsets. However, the ```leukemia``` dataset has very small number of observations, and therefore we keep just a small subset for the test such as $30$%, and use the rest for training. We also scale the data before training. 

In [3]:
scaler = StandardScaler()
X_ = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.3)
print('Train: {}, Test: {}'.format(X_train.shape[0], X_test.shape[0]))

Train: 50, Test: 22


## Classification using Logistic regression

Let us first try to classify the dataset using a classic logistic regression, that we will then use as a baseline. We will compare the performance as well as the usage of the features of the logistic regression with the results of regularized regressions. Let $y_i = \{0,1\}$ be the variable we want to predict for an observation $i$ (in our case the type of leukemia) and $x_i = (x_i^{(1)}, \dots, x_i^{(p)})$ the associated features. The probability for an observation $i$ to belong to either class $0$ or class $1$ is given by

$$p(y_i|x_i, \beta) =  p_i^{y_i}(1-p_i)^{1- y_i}$$

With $p_i = \frac{1}{1+\exp(-\beta^Tx_i)}$. We are mostly interested in estimating $\beta \in \mathbb{R}^p$ to be able to predict class of new data points. The traditional approach to logistic regression is through maximum likelihood. The log-likelihood of logistic regression is given by

$$ \ell(\beta) = \sum_{i=1}^n y_i \log p_i + (1- y_i) \log (1 - p_i)$$

The maximum likelihood estimator $\hat{\beta}$, if it exists, is simply the solution to the problem

$$ \hat{\beta} = \underset{\beta \in \mathbb{R}^p}{\operatorname{argmax}} \ell(\beta) $$

For the implementation we will use the ```LogisticRegression``` function from the ```sklearn``` python library, where we set ```penalty='none'```.

In [4]:
clf_logistic = LogisticRegression(penalty = 'none', solver = 'saga', max_iter=10000)
clf_logistic.fit(X_train, y_train)
print('Accuracy on train set: {:.2f}%'.format(clf_logistic.score(X_train, y_train)*100))
print('Accuracy on test set: {:.2f}%'.format(clf_logistic.score(X_test, y_test)*100))
print('Number of positive coefficients: ', np.sum(clf_logistic.coef_>0))
print('Number of negative coefficients: ', np.sum(clf_logistic.coef_<0))
print('Number of null coefficients: ', np.sum(clf_logistic.coef_==0))

Accuracy on train set: 100.00%
Accuracy on test set: 86.36%
Number of positive coefficients:  3666
Number of negative coefficients:  3463
Number of null coefficients:  0


The logistic regression performs pretty well, but all of the $7129$ features are used in the classification. It may be useful to select the most relevant ones by using regularized methods such as LASSO or Elastic Net.

## Classification using Elastic Net regularization

The LASSO method we have seen in class uses a penalty function based on $ \Vert\beta\Vert_1 = \sum_{i=1} ^p |\beta_i|$. The initial problem of the logistic regression becomes 

$$\hat{\beta} = \underset{\beta \in \mathbb{R}^p}{\operatorname{argmax}} \sum_{i=1}^n \left[ y_i \log p_i + (1- y_i) \log (1 - p_i) \right] - \lambda \Vert\beta\Vert_1 $$

with $\lambda >0$. However, one of the limitations of the LASSO method is that the solution has at most $\min(n,p)$ non zero coefficients. For example, in the $p>>n$ case (our case), the LASSO selects at most $n$ variables before it saturates. In addition, if there is a group of highly correlated variables, then the LASSO tends to select one variable from a group and ignore the others. The Elastic Net method overcomes these problems by adding a quadratic part to the penalty. In reality, the Elastic Net is a regularized regression method that linearly combines the $l1$ and $l2$ penalties of the lasso and ridge methods. The problem becomes 

$$\hat{\beta} = \underset{\beta \in \mathbb{R}^p}{\operatorname{argmax}} \sum_{i=1}^n \left[ y_i \log p_i + (1- y_i) \log (1 - p_i)\right] - \lambda_1 \Vert\beta\Vert_1 - \lambda_2 \Vert\beta\Vert_2^2$$

with $\lambda_1,\lambda_2 >0$. We can notice, when the $l2$ penalty is used alone this brings us back to the Ridge regression seen in class. This new solution can be sparse, but can also have as many non zero coefficients as it wants. In addition, the new problem is strictly convex and there is a grouping effect, i.e. if two features are very similar the elastic net is quite likely to select them both, unlike the LASSO. This new problem also requires to select the right coefficients $\lambda_1$ and $\lambda_2$. 

## Application of the Elastic Net method

For the implementation, we will use the ```LogisticRegressionCV``` function from ```sklearn```, where we set ```penalty='elasticnet'```. In reality, to perform an Elastic Net regression with ```sklearn``` we have two hyper parameters to fine tune, the ```C```parameter and the ```l1_ratio```parameter. Let us consider the Elastic Net penalty function as implemented in the ```LogisticRegressionCV``` function:

$$ \mathcal{C}\left(\lambda \Vert\beta\Vert_1 + (1-\lambda)\Vert\beta\Vert_2^2\right)$$

with $\lambda \in [0,1]$, and $\mathcal{C} > 0$. The hyper parameter ```C``` is describing the inverse of the regularization strength, and the ```l1_ratio``` actually corresponds to the Elastic Net mixing parameter.

#### The ```l1_ratio``` hyperparameter
- A value of 0 is equivalent to using ```penalty='l2'```, which corresponds to the Ridge regression.
- A value of 1 is equivalent to using ```penalty='l1'```, which corresponds to the LASSO regression.
- A value of 0.5 is equivalent to having perfect balance between $l_1$ and $l_2$ penalties.

#### The ```C``` hyperparameter
- A small $\mathcal{C}$ enforces a bigger restriction on the number of non zero coefficients, as it corresponds to a stronger regularization. In other terms, lowering $\mathcal{C}$ strengthens the $\lambda$ regulator, and thus shrinks the number of non zero coefficients.
- A big $\mathcal{C}$ relaxes the restriction on the number of non zero coefficients. 

Let us first try with the values ```C=1``` and ```l1_ratio=0.5```. This setup corresponds to a balanced ratio between $l_1$ and $l_2$ penalties, and a neutral regularization strength.   



In [5]:
clf_elasticnet = LogisticRegression(penalty = 'elasticnet', solver = 'saga', C = 1, max_iter=10000, l1_ratio= 0.5)
clf_elasticnet.fit(X_train, y_train)
print('Accuracy on train set: {:.2f}%'.format(clf_elasticnet.score(X_train, y_train)*100))
print('Accuracy on test set: {:.2f}%'.format(clf_elasticnet.score(X_test, y_test)*100))
print('Number of positive coefficients: ', np.sum(clf_elasticnet.coef_>0))
print('Number of negative coefficients: ', np.sum(clf_elasticnet.coef_<0))
print('Number of null coefficients: ', np.sum(clf_elasticnet.coef_==0))

Accuracy on train set: 100.00%
Accuracy on test set: 90.91%
Number of positive coefficients:  112
Number of negative coefficients:  81
Number of null coefficients:  6936


With the values for the hyper parameters that we chose, we can already see the Elastic Net yields a higher performance than the regular logistic regression  by scoring $90$% accuracy, and selects way less parameters. Indeed, the regularization allows us to have just a little more than $200$ non zero coefficients out of over $7000$, while the regular regression used all of them. 

## Fine tuning of the hyper parameters using $k$-fold Cross Validation

We are now interested in the impact of the hyper parameters' tuning on these results. A solution to choose these hyperparameters properly would be to perform a grid search using $k$-fold cross-validation. In this process, we define a range of values to be tested for ```C``` and ```l1_ratio```, we partition the training data in $k$ equals parts, and each of the parts is then set aside at turn as validation set while classification is performed on the other $k-1$ subsets using each one of the possible values for ```l1_ratio```and ```C```. The values selected are the ones for which the accuracy of the task is maximized. 

We try $10$ different values for ```C``` between $\log(-10)$ and $\log(10)$, $10$ values for ```l1_ratio``` in $[0,1]$, and select the values maximizing the accuracy score using $5$-fold cross-validation.


In [6]:
clf_cv = LogisticRegressionCV(cv = 5, penalty = 'elasticnet', solver = 'saga', Cs = np.logspace(-10,10,10), max_iter=10000, l1_ratios=np.linspace(0,1,10) )
clf_cv.fit(X_train, y_train)
print("Selected regularization strength: ", clf_cv.C_[0])
print("Selected L1 ratio : ", clf_cv.l1_ratio_[0])
print('Accuracy on train set: {:.2f}%'.format(clf_cv.score(X_train, y_train)*100))
print('Accuracy on test set: {:.2f}%'.format(clf_cv.score(X_test, y_test)*100))
print('Number of positive coefficients: ', np.sum(clf_cv.coef_>0))
print('Number of negative coefficients: ', np.sum(clf_cv.coef_<0))
print('Number of null coefficients: ', np.sum(clf_cv.coef_==0))

Selected regularization strength:  0.07742636826811278
Selected L1 ratio :  0.1111111111111111
Accuracy on train set: 100.00%
Accuracy on test set: 95.45%
Number of positive coefficients:  150
Number of negative coefficients:  114
Number of null coefficients:  6865


With the right choice of hyper parameters we reach an even higher accuracy of $95$%, although we notice this time we have a higher number of non null coefficients. There are two elements to take into account to explain this:
- The selected ```C``` is equal to $0.077$. We know a small value for this hyper parameter means a stronger regularization, and therefore in theory we should have less non null coefficients than we had previously, as $0.077 < 1$. 
- However, the selected ```l1_ratio```  is equal to $0.11$. The reason is actually related to this ratio. Indeed, in the previous experiment, we chose ```l1_ratio=0.5``` which corresponded to having the same amount of $l1$ and $l2$ penalty. Here,  having ```l1_ratio=0.11``` corresponds to having a very strong $l2$ penalty, bringing the model closer to the Ridge regression. It makes perfect sense to have more non null coefficients with an important $l2$ penalty as the whole purpose of adding it was to overcome the problem we had with only the $l1$ penalty, by allowing the model to have as many non zero coefficients as it wants.


## Conclusion

We performed a binary classification on the high dimensional ```leukemia``` dataset using an Elastic Net regularized logistic regression. We compared the performance of the Elastic Net with a regular logistic regression and observed a significative improvement in performance as the accuracy on the task increased by almost $10$%. Even more importantly, the Elastic Net allows to reduce the number of features used in the classification by selecting only the relevant ones, and in our application we went from $7129$ initial features to approximately $260$. The main advantage of the Elastic Net over the LASSO is that it overcomes the restriction on the maximum number of non null coefficients the model can select, allowing the model to perform even better without this constraint. 

This second aspect is particularly relevant in the medical domain where $p>>n$ datasets are common, and where it can be extremely useful to perform classification using the less information possible. 